In [1]:
# testing out the logits thing here
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "1,2,3"
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('LLM_IPD')

In [2]:
from utils import *

In [3]:
print_all_models()

google/gemma-7b
aegunal/FT_IPD_gemma7b
mistralai/Mistral-7B-v0.1
aegunal/FT_IPD_mistral7b
meta-llama/Llama-2-7b-hf
aegunal/FT_IPD_llama7b
tiiuae/falcon-7b


In [3]:
# initialize models
model2 = PDModel(model_name_in="mistralai/Mistral-7B-v0.1",
                 player_id_in="2")
model1 = PDModel(model_name_in="mistralai/Mistral-7B-v0.1",
                 player_id_in="1")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
num_rounds = 20
num_games = 10
sv_game_history = {}

for game_id in range(num_games):
    # game_history = ""
    sv_game_history[game_id] = {}
    logger.info("Game: " + str(game_id))
    for round_id in range(num_rounds):
        logger.info("Round: " + str(round_id))

        logger.info("generating model 1")
        model1_gen = model1.next_action_hf()['action']
        logger.info("generating model 2")
        model2_gen = model2.next_action_hf()['action']

        logger.info("model 1, model 2 actions: " + model1_gen + ',' + model2_gen)
        # game_history += "Round " + str(round_id) + ": " + "Player 1 played " + model1_gen + \
        #                  ", Player 2 played " + model2_gen + ". "
        # save game hist
        sv_game_history[game_id][round_id] = {'p1':model1_gen,'p2':model2_gen}
        logger.info("Round " + str(round_id) + " results: " + "P1//" + str(model1_gen) + ", P2//" + str(model2_gen))
        # update game histories for individual models
        model1.add_to_hist(round_id, model1_gen, model2_gen)
        model2.add_to_hist(round_id, model2_gen, model1_gen)
        

INFO:LLM_IPD:Game: 0
INFO:LLM_IPD:Round: 0
INFO:LLM_IPD:generating model 1
/home/gunala/miniconda3/envs/QLORAFTEnv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/gunala/miniconda3/envs/QLORAFTEnv/lib/python3.11/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
INF

In [5]:
import json
output_dirname = "/home/gunala/LLMDialGen/games/scripts/prompt3_scoresmethod/"
output_fname = "mistral7b_VS_falcon7B"
with open(output_dirname + output_fname,'w+') as outf:
    json.dump(sv_game_history,outf)